In [46]:
from __future__ import division
import numpy as np
import sys

In [47]:
# Load input data
X_train = np.genfromtxt('X_train_C.csv', delimiter=',')
y_train = np.genfromtxt('Y_train_C.csv', delimiter=',')
X_test = np.genfromtxt('X_test_C.csv', delimiter=',')

In [49]:
# store number of unique training labels and their frequency
Y, n_y = np.unique(y_train, return_counts=True)
print(list(zip(Y, n_y)))

In [51]:
# Learn prior probablity (pi_hat), MLE for class conditional mean (mu_hat) & class conditional covariance (sigma_hat)
pi_hat = []
mu_hat = []
sigma_hat = []
for y, count_y in zip(Y, n_y):
    
    pi_hat_y = count_y/np.sum(n_y) # class prior
    pi_hat.append(pi_hat_y)
    mu_hat_y = np.dot((y_train == y), X_train)/count_y #class conditional mean
    mu_hat.append(mu_hat_y)
    X_train_y = X_train[y_train.astype(int) == np.int(y), :] #subetting X_train by class
    
    x_mu_hat_dev_y = X_train_y - mu_hat_y
        
    sigma_hat_y = np.dot(x_mu_hat_dev_y.T, x_mu_hat_dev_y)/(count_y) #class conditional covariance since this is QDA
    
    sigma_hat.append(sigma_hat_y)

In [74]:
# predicting for X_test
# calculate QDA probability disctribution for each test sample
y_test = []
    
for x in X_test:
    
    f_x_hat = []
    for y in Y:
        idx = np.int(y)
        x_mu_hat_dev_y = x - mu_hat[idx]

        coefficient = pi_hat[idx]/np.sqrt(np.linalg.det(sigma_hat[idx]))
        
        power_value = -(1/2)*(x_mu_hat_dev_y.T@np.linalg.inv(sigma_hat[idx])@x_mu_hat_dev_y)
 
        f_x_hat_y = coefficient*np.exp(power_value) # PDF for QDA
        f_x_hat.append(f_x_hat_y)

    y_test.append(f_x_hat/np.asarray(f_x_hat).sum()) #probablity disctribution for each class

(42, 10)


In [75]:
#Save output
np.savetxt("probs_test.csv", np.asarray(y_test), delimiter=",") # write output to file